Importing Libraries



In [288]:
# Importing Libraries 
import numpy as np
import pandas as pd
import ast
import nltk
import pickle
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [289]:
#Data
credits =pd.read_csv('DataSet/TMDB 5000/tmdb_5000_credits.csv')
movies=pd.read_csv('DataSet/TMDB 5000/tmdb_5000_movies.csv')

In [290]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [291]:
credits.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [292]:
# Merge dataSet
df=movies.merge(credits,on='title')

# Selecting the Columns we need
df=df[['movie_id','genres', 'keywords', 'title', 'overview', 'cast', 'crew']]

# Missing values
df.isnull().sum()

# Drop null values
df.dropna(inplace=True)

In [293]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 4806 entries, 0 to 4808
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  4806 non-null   int64 
 1   genres    4806 non-null   object
 2   keywords  4806 non-null   object
 3   title     4806 non-null   object
 4   overview  4806 non-null   object
 5   cast      4806 non-null   object
 6   crew      4806 non-null   object
dtypes: int64(1), object(6)
memory usage: 300.4+ KB


In [294]:
df['genres'][0]

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [295]:

def GenresTwo(obj):
    list=[]
    genres=ast.literal_eval(obj)

    for i in genres[:1]:
        list.append(i['name'])
    return list



def Genres(obj):
    list=[]

    for i in ast.literal_eval(obj):
        list.append(i['name'])
    return list


# Function to extract 'name' data and store as a list from keywords and genres
def Keywords(obj):
    list=[]
    for i in ast.literal_eval(obj):
        list.append(i['name'])
    return list

# Function to extract  Actor  Name for first 3 Character
def Actor(obj):
    list=[]
    counter =0
    for i in ast.literal_eval(obj):
        if counter !=3:
            list.append(i['name'])
            counter+=1
        else:
            break
    return list

# Function to extract Director Name
def Director(obj):
    list=[]
    for i in ast.literal_eval(obj):
        if i['job'] =='Director':
            list.append(i['name'])
            break
    return list


df['genresTwo'] = df['genres'].apply(GenresTwo)
df['genres']=df['genres'].apply(Genres)
df['keywords']=df['keywords'].apply(Keywords)
df['cast']=df['cast'].apply(Actor)
df['crew']=df['crew'].apply(Director)

In [296]:
df['overview']=df['overview'].apply(lambda x:x.split())


In [297]:
# Remove the space from each word Example James Cameroon = JamesCameroon
df['genresTwo']=df['genresTwo'].apply(lambda x:[i.replace(" ","") for i in x])
df['genres']=df['genres'].apply(lambda x:[i.replace(" ","") for i in x])
df['keywords']=df['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
df['cast']=df['cast'].apply(lambda x:[i.replace(" ","") for i in x])
df['crew']=df['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [298]:
df.head()

,movie_id,genres,keywords,title,overview,cast,crew,genresTwo
0,19995,"[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...",Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],[Action]
1,285,"[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...",Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski],[Adventure]
2,206647,"[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...",Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes],[Action]
3,49026,"[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...",The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan],[Action]
4,49529,"[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...",John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton],[Action]


In [299]:
df['tags']  =   df['overview']  +   df['cast']  +   df['crew']  +   df['keywords'] + df['genres']
df=df[['movie_id', 'title', 'tags', 'genresTwo']]

# Convert tags from list to String
df['tags']=df['tags'].apply(lambda x:" ".join(x))
df['genresTwo']=df['genresTwo'].apply(lambda x:" ".join(x))


# UpperCase to LowerCase of Tags
df['tags']=df['tags'].apply(lambda x:x.lower())
df['genresTwo']=df['genresTwo'].apply(lambda x:x.lower())


In [300]:
df.head()

,movie_id,title,tags,genresTwo
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di...",action
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believed to be dead, ha...",adventure
2,206647,Spectre,a cryptic message from bond’s past sends him o...,action
3,49026,The Dark Knight Rises,following the death of district attorney harve...,action
4,49529,John Carter,"john carter is a war-weary, former military ca...",action


In [301]:
# Making seperate columns for Genres
df['happy'] = np.nan
df['angry'] = np.nan
df['sad'] = np.nan
df['neutral'] = np.nan

In [302]:
def unique(list1):

    # initialize a null list
    unique_list = []

    # traverse for all elements
    for x in list1:
        # check if exists in unique_list or not
        if x not in unique_list:
            unique_list.append(x)
    # print list
    for x in unique_list:
        print (x)


unique(df['genresTwo'])



action
adventure
fantasy
animation
sciencefiction
drama
thriller
family
comedy
history
war
western
romance
crime
mystery
horror
documentary
music
tvmovie

foreign


In [303]:
import pandas as pd

# Genre mappings
genre_mappings = {
    'sad': ['comedy', 'animation', 'family', 'drama', 'music'],
    'happy': ['fantasy', 'sciencefiction', 'drama', 'thriller', 'western', 'action', 'adventure', 'foreign'],
    'neutral': ['action', 'adventure', 'thriller', 'fantasy', 'sciencefiction', 'history', 'war', 'crime', 'mystery', 'horror', 'documentary', 'tv movie'],
    'angry': ['drama', 'family', 'romance', 'music']

}

# Function to encode emotions based on genres
def encode_emotions(row):
    genre = row['genresTwo'].lower()
    for emotion, genres in genre_mappings.items():
        if genre in [g.lower() for g in genres]:
            row[emotion] = 1
    return row

# Apply the function to the DataFrame
df = df.apply(encode_emotions, axis=1)



In [304]:
df.head()

,movie_id,title,tags,genresTwo,happy,angry,sad,neutral
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di...",action,1.0,NaN,NaN,1.0
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believed to be dead, ha...",adventure,1.0,NaN,NaN,1.0
2,206647,Spectre,a cryptic message from bond’s past sends him o...,action,1.0,NaN,NaN,1.0
3,49026,The Dark Knight Rises,following the death of district attorney harve...,action,1.0,NaN,NaN,1.0
4,49529,John Carter,"john carter is a war-weary, former military ca...",action,1.0,NaN,NaN,1.0


In [305]:
# Stemming
ps=PorterStemmer()

# Function performing Stemming on each tags 
def stemming(text):
    list=[]

    for i in text.split():
        list.append(ps.stem(i))
    return " ".join(list)

# Change for all the values
df['tags']=df['tags'].apply(stemming)

In [306]:
# Splitting dataframe based on the emotion
df_happy = df[df['happy'].notna()].reset_index(drop=True)
df_sad  =df[df['sad'].notna()].reset_index(drop=True)
df_angry = df[df['angry'].notna()].reset_index(drop=True)
df_neutral = df[df['neutral'].notna()].reset_index(drop=True)



# Convert tags to vectors
cv = CountVectorizer(max_features=5000, stop_words='english')

vectors_happy = cv.fit_transform(df_happy['tags']).toarray()
vectors_sad = cv.fit_transform(df_sad['tags']).toarray()
vectors_angry = cv.fit_transform(df_angry['tags']).toarray()
vectors_neutral = cv.fit_transform(df_neutral['tags']).toarray()

# Calculate cosine similarity
similarity_happy = cosine_similarity(vectors_happy)
similarity_sad = cosine_similarity(vectors_sad)
similarity_angry = cosine_similarity(vectors_angry)
similarity_neutral = cosine_similarity(vectors_neutral)


In [307]:

def display_random_movie(emotion):
    filtered_movie = df[df[emotion].notna()]
    return filtered_movie['title'].sample(n=5)

def recommend(movie, similarity_matrix, df):
    index = df[df['title'] == movie].index[0]
    distances = sorted(enumerate(similarity_matrix[index]), reverse=True, key=lambda x: x[1])

    for i in distances[1:6]:
        recommended_movie = df.iloc[i[0]].title
        similarity_score = i[1]
        print(f"{recommended_movie} (Cosine Similarity: {similarity_score:.3f})")
    
    

def main():
    emotion = input('What are emotion are you feeling').strip().lower()
    if emotion not in ['happy', 'angry', 'neutral', 'sad']:
        print('Invalid Emotion Selected')
        return
    
    if emotion == 'happy':
        similarity_matrix = similarity_happy
        df_emotion = df_happy

    elif emotion == 'angry':
        similarity_matrix = similarity_angry
        df_emotion = df_angry

    elif emotion == 'neutral':
        similarity_matrix = similarity_neutral
        df_emotion = df_neutral

    elif emotion == 'sad':
        similarity_matrix = similarity_sad
        df_emotion = df_sad


    
    random_movies = display_random_movie(emotion)
    print(random_movies)
    select_movie_from_list = input('Select the Movie from the Option')
    print('Many thanks for choosing Movie -', select_movie_from_list)
    print()

    recommend(select_movie_from_list, similarity_matrix, df_emotion)

main()

2646    Things to Do in Denver When You're Dead
3065                             The Red Violin
1798                                Pain & Gain
2052                                  RoboCop 3
3740                                Night Watch
Name: title, dtype: object
Many thanks for choosing Movie - Batman

Batman (Cosine Similarity: 0.944)
Batman & Robin (Cosine Similarity: 0.406)
Batman Returns (Cosine Similarity: 0.333)
Batman Begins (Cosine Similarity: 0.327)
Punisher: War Zone (Cosine Similarity: 0.291)


In [309]:
# # #
# # #! To check the recommend function
# df_happy = df[df['happy'].notna()].reset_index(drop=True)



# cv_happy=CountVectorizer(max_features = 5000, stop_words='english')
# vectors_happy = cv_happy.fit_transform(df_happy['tags']).toarray()

# similarity_happy = cosine_similarity(vectors_happy)


# def recommend(movie):
#     index = df_happy[df_happy['title'] == movie].index[0]
#     distances = sorted(enumerate(similarity_happy[index]), reverse=True, key=lambda x: x[1])

#     for i in distances[1:6]:
#         recommended_movie = df_happy.iloc[i[0]].title
#         similarity_score = i[1]
#         print(f"{recommended_movie} (Cosine Similarity: {similarity_score:.3f})")

In [317]:
recommend('')

IndexError: index 0 is out of bounds for axis 0 with size 0